<a href="https://colab.research.google.com/github/1337Rinz/img_retrieval_vectorDB/blob/main/build_vector_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#faiss


Facebook AI Similarity Search (Faiss) là một thư viện sử dụng similiarity search cùng với clustering các vector.

In [1]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.7 MB/s eta 0:00:00


In [2]:
import faiss
import numpy as np

##Tạo một vectorDB và thêm dữ liệu vào



In [3]:
# Tạo một bộ dữ liệu giả
d = 128                           # Chiều của vector
nb = 1000                         # Số lượng vector trong bộ dữ liệu
np.random.seed(1234)              # Đặt seed cho tính ngẫu nhiên
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.

# Tạo index
index = faiss.IndexFlatL2(d)      # L2 distance index
print(index.is_trained)
index.add(xb)                     # Thêm vector vào index
print(index.ntotal)

True
1000


##Tìm kiếm các vector tương tự

In [4]:
# Tạo các vector cần tìm kiếm
nq = 10                           # Số lượng vector truy vấn
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

# Tìm kiếm
k = 4                             # Số lượng vector gần nhất cần tìm
D, I = index.search(xq, k)        # D là khoảng cách, I là chỉ số của vector gần nhất

print(I)
print(D)


[[631 345 947 331]
 [568 443 933 583]
 [540 462 349 901]
 [694 131 564  92]
 [ 94 477  61  42]
 [319 444 792 182]
 [671  62 693 485]
 [263  49 546 552]
 [930 214  97 835]
 [266 655 206 271]]
[[13.80421   14.963522  15.179419  15.210806 ]
 [14.912462  15.244122  15.992789  15.995716 ]
 [15.650552  16.897297  17.99605   18.100945 ]
 [14.553017  14.594555  14.786146  14.819241 ]
 [16.726444  16.758434  16.829638  16.939337 ]
 [12.676683  15.317959  15.351387  15.539641 ]
 [15.94039   16.315487  16.320164  16.892683 ]
 [13.477825  14.249644  14.252818  14.480717 ]
 [11.693054  15.345835  15.357733  15.582954 ]
 [14.892674  15.7037735 16.854427  16.984077 ]]


#annoy

Annoy là một thư viện được phát triển bởi Spotify, sử dụng để tìm kiếm vector tương tự bằng cách sử dụng các cây tìm kiếm (trees). Annoy rất hiệu quả cho các ứng dụng với dữ liệu lớn và có thể sử dụng bộ nhớ ít hơn.



In [5]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552452 sha256=e3c0ef60a10d1663f0396e398b858908557641439a305100dddb4621f1cae093
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


##Tạo một vectorDB và thêm dữ liệu vào

In [6]:
from annoy import AnnoyIndex
import random

f = 128                          # Chiều của vector
t = AnnoyIndex(f, 'angular')     # Angular distance index

# Thêm dữ liệu vào index
for i in range(1000):
    v = [random.gauss(0, 1) for z in range(f)]
    t.add_item(i, v)

t.build(10)                      # Xây dựng 10 cây cho việc tìm kiếm
t.save('test.ann')

True

##Tìm kiếm các vector tương tự

In [7]:
u = AnnoyIndex(f, 'angular')
u.load('test.ann')

# Tìm 10 vector gần nhất với vector có chỉ số 0
print(u.get_nns_by_item(0, 10))

[0, 650, 450, 419, 749, 582, 363, 325, 122, 342]
